Created on Tue Aug 27 22:21:22 2024

@author: Santiago D'hers

## Manage h5 files

### Use:

- This notebook will take .H5 files (from DeepLabCut) and prepare the position.csv files to be analyzed.

- It filters out low likelihood positions, interpolates and smoothens the data.

- The positions are also scaled from pixels to cm for better generalization.


### Requirements:

- A folder with files of extention .H5 (from DeepLabCut) containing:

    - The position of the desired bodyparts and objects on the video.

    - The position filenames should contain the trial name, it will be used to organize the files into subfolders.

---
#### 1. Load the necessary modules

In [1]:
import os
from glob import glob
import STORM_functions as storm

---
#### 2. Set your working directory and experiment information, and specify thresholds.
`base` : The directory path where we can find the folder containing the position files. Note that if you are using a Windows path with backslashes, place an ‘r’ in front of the directory path to avoid an error (e.g., r'C:\Users\dhers\OneDrive - UBA\workshop').

`experiment` : The name of the experiment folder, where your position files are stored.

`filter_example` : A word that will be used to filter the position file used as an example.

`trials` : If your experiment has multiple trials, specify the trial names here.

`confidence` : State how many std_dev away from the mean the points can be without being erased (it is similar to asking "how good is your tracking?").

`tolerance` : If the mean likelihood is below this value, the whole point will be erased (because it is probably not there).

`median_filter` : State how many frames to use for the median filter. It must be an odd number.

`example_bodypart` : State which bodypart you'd like to plot as an example.

`objects` : Name the stationary objects that may appear on your data.

`measured_points` : Measure the distance between two bodyparts to scale the video from pixels to cm. I use the left and right ears.

`measured_dist` : State the distance between the measured points. Distance between the ears is 1.8 cm in my c57 mice.

`video_fps` : State the frames per second of the videos.

In [2]:
# State your path:
base = r'C:\Users\dhers\OneDrive - UBA\workshop'
experiment = r'TOM'
path = os.path.join(base, experiment)

all_h5_files = glob(os.path.join(path,"*position.h5"))
filter_example = 'TS'

trials  = ["Hab", "TR1", "TR2", "TS"]

confidence = 2
tolerance = 0.5
median_filter = 3 

example_bodypart = 'nose'
objects = ['obj_1', 'obj_2']

measured_points = ['L_ear', 'R_ear']
measured_dist = 1.8 # in cm

video_fps = 30

---
#### 3. We can open the h5 file and see what is inside

In [3]:
# Choose an example file to plot:
example_path = storm.choose_example(all_h5_files, filter_example)

ValueError: The list of files is empty. Please provide a non-empty list.

In [ ]:
# Open the example file:
df_raw, bodyparts = storm.open_h5_file(example_path, print_data=True, num_sd=confidence)

Positions obtained by model: DLC_resnet50_SauronSep30shuffle1_200000
Points in df: ['L_ear', 'R_ear', 'body', 'head', 'neck', 'nose', 'obj_1', 'obj_2', 'tail_1', 'tail_2', 'tail_3']
L_ear 	 median: 1.00 	 mean: 0.98 	 std_dev: 0.14 	 tolerance: 0.70
R_ear 	 median: 1.00 	 mean: 0.98 	 std_dev: 0.14 	 tolerance: 0.70
body 	 median: 1.00 	 mean: 0.98 	 std_dev: 0.14 	 tolerance: 0.71
head 	 median: 1.00 	 mean: 0.98 	 std_dev: 0.14 	 tolerance: 0.70
neck 	 median: 1.00 	 mean: 0.98 	 std_dev: 0.14 	 tolerance: 0.70
nose 	 median: 1.00 	 mean: 0.95 	 std_dev: 0.19 	 tolerance: 0.58
obj_1 	 median: 1.00 	 mean: 1.00 	 std_dev: 0.00 	 tolerance: 1.00
obj_2 	 median: 1.00 	 mean: 0.90 	 std_dev: 0.28 	 tolerance: 0.35
tail_1 	 median: 1.00 	 mean: 0.98 	 std_dev: 0.15 	 tolerance: 0.68
tail_2 	 median: 1.00 	 mean: 0.97 	 std_dev: 0.15 	 tolerance: 0.67
tail_3 	 median: 1.00 	 mean: 0.96 	 std_dev: 0.17 	 tolerance: 0.63


---
#### 4. Now that we have it open, lets test our processing parameters in an example video

In [ ]:
df_smooth = storm.filter_and_smooth_df(df_raw, bodyparts, objects, med_filt_window = 3, drop_below = tolerance, num_sd = confidence)

In [ ]:
storm.plot_raw_vs_smoothed(df_raw, df_smooth, bodypart = example_bodypart, num_sd = confidence)

---
#### 5. (Optional) If we want to scale our data from pixels to cm, we can use a measured distance between two bodyparts (e.g. the distance between both ears).

In [ ]:
# plot scale
scale = storm.find_scale_factor(df_smooth, measured_dist, measured_points, print_results=True)

median distance is 34.87384847503316, mean distance is 34.87902323263705. Scale factor is 0.0516 (1 cm = 19.37 px).


---
#### 6. Now that we know what we are doing, we can apply all previous steps to all the files in our folder and store the results into csv files (lets face it, they are less scary).

In [ ]:
# Process every file in the folder
storm.process_position_files(all_h5_files, objects, measured_dist, measured_points, scale = True, fps = video_fps, med_filt_window = median_filter, drop_below = tolerance, num_sd = confidence)

2023-05_TORM-2m-24h_Hab_C1_A_L_position.h5 has 18 columns. The mouse took 1.93 sec to enter. Scale factor is 0.0524 (1 cm = 19.09 px).
2023-05_TORM-2m-24h_Hab_C1_A_R_position.h5 has 18 columns. The mouse took 2.80 sec to enter. Scale factor is 0.0528 (1 cm = 18.95 px).
2023-05_TORM-2m-24h_Hab_C1_B_L_position.h5 has 18 columns. The mouse took 1.87 sec to enter. Scale factor is 0.0515 (1 cm = 19.42 px).
2023-05_TORM-2m-24h_Hab_C1_B_R_position.h5 has 18 columns. The mouse took 3.60 sec to enter. Scale factor is 0.0511 (1 cm = 19.58 px).
2023-05_TORM-2m-24h_Hab_C2_A_L_position.h5 has 18 columns. The mouse took 1.77 sec to enter. Scale factor is 0.0500 (1 cm = 19.99 px).
2023-05_TORM-2m-24h_Hab_C2_A_R_position.h5 has 18 columns. The mouse took 0.97 sec to enter. Scale factor is 0.0529 (1 cm = 18.91 px).
2023-05_TORM-2m-24h_Hab_C2_B_L_position.h5 has 20 columns. The mouse took 6.27 sec to enter. Scale factor is 0.0521 (1 cm = 19.19 px).
2023-05_TORM-2m-24h_Hab_C2_B_R_position.h5 has 18 colum

---
#### 7. Finally, we can organize the files into subfolders corresponding to different trials of the experiment.

In [ ]:
# Clean the folder
storm.filter_and_move_files(path, trials)

Files filtered and moved successfully.
All .H5 files are stored away
